In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [ ]:
df = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='出貨明細')

### Dash pivottable資料前處理


In [ ]:
df['BG'] = df.apply(lambda x: 'RFBU2' if 'RFDP' in x['品名'] else 'RFBU1' if 'RFDP' not in x['品名'] and 'RF' in x['BG'] else x['BG'], axis=1)

In [ ]:
df = df[df['狀態'].str.contains('出')]

In [ ]:
df = df[['BG', 'Group', '銷售單號', '開單日期', '預交日期', '預交年份', '預交月份', '負責業務', '品名', '幣別', '單價', '數量', '本國幣別NTD', '客戶料號', 'Term']]

In [ ]:
mon_dict = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
df['數量'] = df['數量'].astype('int')
df['本國幣別NTD'] = df['本國幣別NTD'].astype('int')
df['預交月份'] = df['預交月份'].map(mon_dict)


### 處理完畢

In [ ]:
df1 = df['出貨明細']
df2 = df['2020預算']

In [ ]:
df2 = df2[df2['負責業務'] == '許凱智'].reset_index(drop=True)

In [ ]:
df2.loc[(df2['客戶名稱'].str.contains('RF')), '類型'] = 'RF'
df2.loc[(df2['類型'] != 'RF'), '類型'] = 'Component'

In [ ]:
df2 # Budget table

In [ ]:
df1.loc[(df1['BG'] != 'RF') , 'BG'] = 'Component'

In [ ]:
df1 = df1[~(df1['狀態'] == '合約')]

In [ ]:
df1 = df1[~(df1['狀態'] == '待通知')]

In [ ]:
df1 = df1.loc[:,['Group', 'BG', '負責業務', '狀態', '預交日期', '本國幣別NTD']]

In [ ]:
df1['預交年份'] = df1['預交日期'].dt.year
df1['預交月份'] = df1['預交日期'].dt.month

In [ ]:
df1 = df1[df1['負責業務']=='許凱智'].reset_index(drop=True)


In [ ]:
df1 = df1.groupby(['Group', 'BG', '預交年份','預交月份'])[['本國幣別NTD']].sum().reset_index() #最終的實績表格

In [ ]:
def month_trans(x):
    dic = {'1': 'January', '2':'February', '3':'March', '4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}
    if x == '1':
        return dic[x]
    elif x == '2':
        return dic[x]
    elif x == '3':
        return dic[x]
    elif x == '4':
        return dic[x]
    elif x == '5':
        return dic[x]
    elif x == '6':
        return dic[x]
    elif x == '7':
        return dic[x]
    elif x == '8':
        return dic[x]
    elif x == '9':
        return dic[x]
    elif x == '10':
        return dic[x]
    elif x == '11':
        return dic[x]
    elif x == '12':
        return dic[x]


In [ ]:
df1['預交月份'] = df1['預交月份'].astype('str')
df1['預交月份'] = df1['預交月份'].map(lambda x: month_trans(x))

In [ ]:
df1.loc[(df1['Group'] == 'MT-SYSTEM') & (df1['BG'] == 'Component'), 'Group'] = 'Mt System(Component)'
df1.loc[(df1['Group'] == 'MT-SYSTEM') & (df1['BG'] == 'RF'), 'Group'] = 'Mt System(RF)'
df1.loc[(df1['Group'] == 'TELTONIKA') & (df1['BG'] == 'Component'), 'Group'] = 'Teltonika(Component)'
df1.loc[(df1['Group'] == 'TELTONIKA') & (df1['BG'] == 'RF'), 'Group'] = 'Teltonika(RF)'
df1.loc[(df1['Group'].str.contains('TE')), 'Group'] = 'Te(RF)'

In [ ]:
df2.loc[(df2['客戶名稱'].str.contains('華科')), '客戶名稱'] = '華新科技'

In [ ]:
df1 = df1.rename(columns={'Group':'客戶名稱','BG':'類型','預交年份':'年份','預交月份':'Month'})

In [ ]:
df2.columns

In [ ]:
result = pd.merge(df1, df2, on=['客戶名稱','類型','年份','Month'], how='left')

In [ ]:
result = pd.read_excel(r'C:\Users\kaihsu\Desktop\Amber需要的每月預算.xlsx')

In [ ]:
result['預估業績'] = pd.to_numeric(result['預估業績'])

In [ ]:
result.info()

In [ ]:
result.pivot_table(values=['預估業績'], index=['客戶名稱', '年份'], columns=['Month'])
result.to_excel(r'C:\Users\kaihsu\Desktop\Amber需要的每月預算1.xlsx')

In [ ]:
result.info()

In [ ]:
result.to_excel(r'C:\Users\kaihsu\Desktop\Amber需要的每月預算.xlsx', index=False)

In [ ]:
result.groupby(['客戶名稱','類型','年份','Month']).agg({'預估業績':'sum', '本國幣別NTD':'sum'})

In [ ]:
df3 = df1.loc[:,['Group', 'BG', '負責業務', '狀態','預交日期', '本國幣別NTD']]

In [ ]:
df3 = df3[df3['預交日期'] >= '2020-01-01'].reset_index(drop=True)

In [ ]:
df3.info()

In [ ]:
df3 = df3[~(df3['狀態'] == '待通知') | (df3['狀態'] == '合約')]

In [ ]:
df3['預交日期'] = df3['預交日期'].dt.month

In [ ]:
df3 = df3.reset_index(drop=True)

In [ ]:
df3.rename(columns={'預交日期':'預交月份'}, inplace=True)

In [ ]:
df4 = df3.groupby(['Group', 'BG','預交月份', '負責業務'])[['本國幣別NTD']].sum().reset_index()

In [ ]:
df4 = df4[df4['負責業務'] == '許凱智']

In [ ]:
df2 = df2[df2['負責業務'] == '許凱智']

In [ ]:
df4['負責業務'] = df4['負責業務'].astype('str')

In [ ]:
df4['預交月份'] = df4['預交月份'].astype('str')

In [ ]:
df4['預交月份'] = df4['預交月份'].map(lambda x: month_trans(x))

In [ ]:
df2.loc[(df2['客戶名稱'].str.contains('Component')), '類型'] = '元件'

In [ ]:
df2['客戶名稱'].value_counts()

In [ ]:
df2.loc[(df2['客戶名稱'].str.contains('華科')), '客戶名稱'] = '華新科技'
df2.loc[(df2['客戶名稱'].str.contains('Te')), '客戶名稱'] = 'TE'


In [ ]:
df2['客戶名稱'].value_counts()

In [ ]:
df4.loc[(df4['Group'].str.contains('MT-')) & (df4['BG'].str.contains('RF')),'Group'] = 'Mt System(RF)'
df4.loc[(df4['Group'].str.contains('MT-')) & (df4['BG'].str.contains('元件')),'Group'] = 'Mt System(Component)'


In [ ]:
df4

In [ ]:
df1['BG'].value_counts()